In [1]:
import pandas as pd


In [2]:
df= pd.read_csv(r"dataset/synthetic_logs.csv")
df.head()

,timestamp,source,log_message,target_label,complexity
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert


In [3]:
df.source.unique()

array(['ModernCRM', 'AnalyticsEngine', 'ModernHR', 'BillingSystem',
       'ThirdPartyAPI', 'LegacyCRM'], dtype=object)

In [4]:
df.target_label.unique()

array(['HTTP Status', 'Critical Error', 'Security Alert', 'Error',
       'System Notification', 'Resource Usage', 'User Action',
       'Workflow Error', 'Deprecation Warning'], dtype=object)

In [5]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN, dbscan
import numpy as np
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(df['log_message'].tolist())
embeddings[:2]

C:\Users\varsv\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\varsv\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\varsv\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to ru

array([[-1.02939621e-01,  3.35459262e-02, -2.20260601e-02,
         1.55105593e-03, -9.86920949e-03, -1.78956255e-01,
        -6.34409785e-02, -6.01761676e-02,  2.81108841e-02,
         5.99620454e-02, -1.72618646e-02,  1.43367471e-03,
        -1.49560004e-01,  3.15285777e-03, -5.66031076e-02,
         2.71685384e-02, -1.49890380e-02, -3.54037546e-02,
        -3.62936258e-02, -1.45410234e-02, -5.61493542e-03,
         8.75539109e-02,  4.55120429e-02,  2.50963680e-02,
         1.00187706e-02,  1.24267014e-02, -1.39923617e-01,
         7.68696442e-02,  3.14095207e-02, -4.15248703e-03,
         4.36902680e-02,  1.71250310e-02, -8.00951496e-02,
         5.74005917e-02,  1.89091805e-02,  8.55262280e-02,
         3.96399237e-02, -1.34371832e-01, -1.44364929e-03,
         3.06707853e-03,  1.76854029e-01,  4.44886740e-03,
        -1.69275180e-02,  2.24266723e-02, -4.35049348e-02,
         6.09028572e-03, -9.98169184e-03, -6.23973049e-02,
         1.07372776e-02, -6.04895549e-03, -7.14660808e-0

In [8]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN, dbscan
import numpy as np
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(df['log_message'].tolist())
dbscan= DBSCAN(eps = 0.2, min_samples=1, metric='cosine')
clusters = dbscan.fit_predict(embeddings)
df['cluster']= clusters
df.head()

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 1513.49it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


,timestamp,source,log_message,target_label,complexity,cluster
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,2
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert,0
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert,0


In [9]:
df[df.cluster == 1].head()

,timestamp,source,log_message,target_label,complexity,cluster
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1
10,8/9/2025 18:58,ModernCRM,Email server encountered a sending fault,Error,bert,1
217,1/22/2025 5:45,BillingSystem,Mail service encountered a delivery glitch,Error,bert,1
248,5/2/2025 23:04,ModernHR,Service disruption caused by email sending error,Critical Error,bert,1
265,3/30/2025 23:53,ModernCRM,Email system had a problem sending emails,Error,bert,1


In [10]:
# python
# sort clusters by size and print 5 messages for clusters with >10 records
cluster_counts = df['cluster'].value_counts()            # descending by default
big_clusters = cluster_counts[cluster_counts > 10]

if big_clusters.empty:
    print("No clusters with more than 10 records.")
else:
    for cluster_label, count in big_clusters.items():
        print(f"\nCluster {cluster_label} — {count} records")
        msgs = df.loc[df['cluster'] == cluster_label, 'log_message'].head(5).tolist()
        for i, msg in enumerate(msgs, 1):
            print(f"{i}. {msg}")


Cluster 0 — 1017 records
1. nova.osapi_compute.wsgi.server [req-b9718cd8-f65e-49cc-8349-6cf7122af137 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2675118
2. nova.osapi_compute.wsgi.server [req-4895c258-b2f8-488f-a2a3-4fae63982e48 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" HTTP status code -  200 len: 211 time: 0.0968180
3. nova.osapi_compute.wsgi.server [req-ee8bc8ba-9265-4280-9215-dbe000a41209 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" RCODE  200 len: 1874 time: 0.2280791
4. nova.osapi_compute.wsgi.server [req-f0bffbc3-5ab0-4916-91c1-0a61dd7d4ec2 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/

In [51]:
# python
import re

def classify_with_regex(log_message):
    regex_patterns = {
        r"User User\d+ logged (in|out).": "User Action",
        r"Backup (started|ended) at .*": "System Notification",
        r"Backup completed successfully.": "System Notification",
        r"System updated to version .*": "System Notification",
        r"File .* uploaded successfully by user .*": "System Notification",
        r"Disk cleanup completed successfully.": "System Notification",
        r"System reboot initiated by user .*": "System Notification",
        r"Account with ID .* created by .*": "User Action",

    }
    for pattern, label in regex_patterns.items():
        if re.search(pattern, str(log_message), re.IGNORECASE):
            return label
    return "Other"

In [52]:
classify_with_regex("account with ID 5351 created by user634.")

'User Action'

In [53]:
df['regex_labels'] = df['log_message'].apply(classify_with_regex)
df.head()
df['regex_labels'].value_counts()

regex_labels
Other                  1910
System Notification     356
User Action             144
Name: count, dtype: int64

In [54]:
df[df.regex_labels.notnull()].head(20)

,timestamp,source,log_message,target_label,complexity,cluster,regex_label,regex_labels
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0,Other,Other
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1,Other,Other
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,2,Other,Other
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert,0,Other,Other
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert,0,Other,Other
5,2025-10-09 10:30:31,ModernHR,nova.osapi_compute.wsgi.server [req-f0bffbc3-5...,HTTP Status,bert,0,Other,Other
6,3/1/2025 19:14,ModernHR,Shard 6 replication task ended in failure,Error,bert,3,Other,Other
7,10/11/2025 8:44,ModernHR,File data_6169.csv uploaded successfully by us...,System Notification,regex,4,System Notification,System Notification
8,2025-02-12 10:42:29,ThirdPartyAPI,nova.compute.claims [req-a07ac654-8e81-416d-bf...,Resource Usage,bert,5,Other,Other
9,2025-03-30 04:01:45,ModernHR,nova.osapi_compute.wsgi.server [req-2bf7cfee-a...,HTTP Status,bert,0,Other,Other


In [55]:
df.shape


(2410, 8)

In [56]:
df[df.regex_labels.notnull()].shape


(2410, 8)

In [57]:
df['regex_labels']=df['log_message'].apply(classify_with_regex)
df[df.regex_label.notnull()].shape


(2410, 8)

In [58]:
df_non_regexes=df[df.regex_label.isnull()].copy()
df_non_regexes

,timestamp,source,log_message,target_label,complexity,cluster,regex_label,regex_labels
